## Fleet Assignment Model

In [4]:
!pip install cvxpy
import numpy as np
import pandas as pd
import cvxpy as cp
import scipy.linalg as la
import matplotlib.pyplot as plt
from scipy.optimize import linprog

### Sets and Indices
- Let $F$ denote the set of flight legs (e.g., IST-YVR, YVR-IST, etc.), indexed by $i \in F$.
- Let $K$ denote the set of aircraft fleet types (e.g., B777-300ER, B787-9), indexed by $j \in K$.
- Let $P$ denote the set of paired (outbound, return) flight legs, where each pair is indexed by $(i, i') \in P$ (e.g., (TK75, TK76)).

### Parameters (Inputs)
- $d_i$: distance of flight leg $i$ (miles)
- $h_i$: block time of flight leg $i$ (hours), computed as
  $$
  h_i = \frac{d_i}{v}
  $$
  where $v$ is an assumed constant cruise speed
- $N_j$: number of available aircraft of fleet type $j$
- $\text{fuel}_j$: fuel burn rate of fleet type $j$ (kg/hour)
- $\text{crew}_j$: crew cost rate of fleet type $j$ ($\$$/hour)
- $\text{maint}_j$: maintenance/other operating cost rate of fleet type $j$ (USD/hour)
- $p_{\text{fuel}}$: fuel price ($\$$/kg)
- $H_{\max}$: maximum usable crew-hours (block hours) per aircraft per day
- $\text{range}_j$: maximum feasible flight range of fleet type $j$ (miles)

Define the per-assignment operating cost $c_{i,j}$ as:
$$
c_{i,j} =
\left(\text{fuel}_j \cdot p_{\text{fuel}} + \text{crew}_j + \text{maint}_j\right)\cdot h_i
$$


In [5]:
flights = pd.DataFrame([
    {"fid":"TK75",  "orig":"IST", "dest":"YVR", "dist_mi":5973.9},
    {"fid":"TK76",  "orig":"YVR", "dest":"IST", "dist_mi":5973.9},

    {"fid":"TK001", "orig":"IST", "dest":"JFK", "dist_mi":5000.0}, 
    {"fid":"TK002", "orig":"JFK", "dest":"IST", "dist_mi":5000.0},

    {"fid":"TK193", "orig":"IST", "dest":"LHR", "dist_mi":1550.0},
    {"fid":"TK194", "orig":"LHR", "dest":"IST", "dist_mi":1550.0},
])


fleet = pd.DataFrame([
    {
        "type":"B777-300ER", "seats":349, "fuel_kg_hr":7500,
        "maint_usd_hr":2200,
        "pilot_usd_hr":1800,
        "cabin_usd_hr":1200,
        "N":2, "range_mi":7600
    },
    {
        "type":"B787-9", "seats":300, "fuel_kg_hr":5600,
        "maint_usd_hr":2000,
        "pilot_usd_hr":1700,
        "cabin_usd_hr":1100,
        "N":2, "range_mi":7600
    },
])

In [6]:
FUEL_PRICE_USD_PER_KG = 0.90
CRUISE_MPH = 560

# daily limit per aircraft
H_MAX = 15.0  # 15 hours/day/aircraft

# flight hours
flights["hours"] = flights["dist_mi"] / CRUISE_MPH

# cost matrix c[i,j]
I, J = len(flights), len(fleet)
C = np.zeros((I, J))

for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        hours = f["hours"]

        fuel_cost = a["fuel_kg_hr"] * hours * FUEL_PRICE_USD_PER_KG
        maint_cost = a["maint_usd_hr"] * hours
        crew_cost = (a["pilot_usd_hr"] + a["cabin_usd_hr"]) * hours

        C[i, j] = fuel_cost + maint_cost + crew_cost

### Decision Variables
- $x_{i,j} \in \{0,1\}$  
  $x_{i,j} = 1$ if flight $i$ is assigned fleet type $j$, and $0$ otherwise.

In [7]:
x = cp.Variable((I, J), boolean=True)

### Constraints

#### 1. Flight Coverage
Each flight leg must be assigned to exactly one fleet type:
$$
\sum_{j \in K} x_{i,j} = 1 \quad \forall i \in F
$$

#### 2. Crew-Hour availability by Fleet Type
Each aircraft of fleet type $j$ is assumed to operate at most $H_{\max}$ crew-hours per day.  
Because each assigned flight would have crew-hours equal to its block time, total crew-hours used by fleet type $j$ cannot exceed available crew-hours:
$$
\sum_{i \in F} h_i\, x_{i,j} \le N_j \cdot H_{\max} \quad \forall j \in K
$$

#### 3. Aircraft Range Feasibility
A fleet type may only be assigned to flight legs within its operational range. If the route distance exceeds the aircraft’s maximum range, that assignment is infeasible:
$$
x_{i,j} = 0 \quad \text{if } d_i > \text{range}_j
$$


#### 4. Operational Pairing (Round-Trip Consistency)
!! Not sure if this is used in real life but seems to add consistency to the model

For paired outbound and return flight legs $(i,i') \in P$, the same fleet type must be used on both legs to ensure operational consistency:

$$
x_{i,j} = x_{i',j} \quad \forall j \in K,\; \forall (i,i') \in P
$$

Can add more here


In [8]:
constraints = []

constraints += [cp.sum(x[i, :]) == 1 for i in range(I)]

pairs = [("TK75", "TK76"), ("TK001", "TK002"), ("TK193", "TK194")]

fid_to_idx = {fid: idx for idx, fid in enumerate(flights["fid"])}

for f1, f2 in pairs:
    i1, i2 = fid_to_idx[f1], fid_to_idx[f2]
    constraints += [x[i1, :] == x[i2, :]]  # same fleet type chosen


# Total crew-hours used by fleet type j = sum_i (hours_i * x_{i,j})
# must be <= (number of available aircraft of type j) * (max crew-hours per aircraft per day)
CREW_H_MAX = H_MAX
for j in range(J):
    crew_hours_used_j = cp.sum(cp.multiply(flights["hours"].values, x[:, j]))
    constraints += [crew_hours_used_j <= int(fleet.loc[j, "N"]) * CREW_H_MAX]

# If distance > range, x[i,j] = 0
for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        if float(f["dist_mi"]) > float(a["range_mi"]):
            constraints += [x[i, j] == 0]

### Objective Function
Minimize total operating cost across all flights and fleet types:
$$
\min \sum_{i \in F}\sum_{j \in K} c_{i,j}\,x_{i,j}
$$


In [9]:
objective = cp.Minimize(cp.sum(cp.multiply(C, x)))

In [10]:
prob = cp.Problem(objective, constraints)
prob.solve()

print("Status:", prob.status)
print("Min cost:", prob.value)

assignments = []
for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))

    assignments.append({
        "flight": f["fid"],
        "route": f"{f['orig']}-{f['dest']}",
        "assigned_fleet": fleet.loc[j_star, "type"],
        "hours": float(f["hours"]),
        "fuel_cost_usd": float(
            fleet.loc[j_star, "fuel_kg_hr"] * f["hours"] * FUEL_PRICE_USD_PER_KG
        ),
        "crew_cost_usd": float(
            (fleet.loc[j_star, "pilot_usd_hr"] + fleet.loc[j_star, "cabin_usd_hr"]) * f["hours"]
        ),
        "total_cost_usd": float(C[i, j_star])
    })

print(pd.DataFrame(assignments))

Status: optimal
Min cost: 477804.19999999995
  flight    route assigned_fleet      hours  fuel_cost_usd  crew_cost_usd  \
0   TK75  IST-YVR         B787-9  10.667679   53765.100000   29869.500000   
1   TK76  YVR-IST         B787-9  10.667679   53765.100000   29869.500000   
2  TK001  IST-JFK     B777-300ER   8.928571   60267.857143   26785.714286   
3  TK002  JFK-IST     B777-300ER   8.928571   60267.857143   26785.714286   
4  TK193  IST-LHR         B787-9   2.767857   13950.000000    7750.000000   
5  TK194  LHR-IST         B787-9   2.767857   13950.000000    7750.000000   

   total_cost_usd  
0   104969.957143  
1   104969.957143  
2   106696.428571  
3   106696.428571  
4    27235.714286  
5    27235.714286  
